![ERA-LOGO](data/logo.png)

## Initialise Python Environment
Run the cell bellow to get everything setup and ready to go! \
This can take some time, so sit back and wait a few minutes...

In [ ]:
from ERA_lib.requirements import install; await install();
from ERA_lib.software import *

## Output View

Instructions:
1. Run the cell bellow
2. Right click on text that says "-- Output View --"
3. Select "Create New View for Cell Output"
4. Drag the new window to the side of the screen (or wherever you like)
5. Click the blue bar to the left of the output below to hide it.

In [ ]:
output_view = widgets.Output()
with output_view:
    print("\n\n-- Output View --\n\n")
output_view

<br />
Run this cell to clear the Output View:

In [ ]:
output_view.clear_output()

<br />

## Upload a file:
Run the cell below, then click the "Select File" button and select your file.

In [ ]:
uploader = FileUploader()

Run the cell below to upload the file:

In [ ]:
uploader.write_file()

## Processing the data:

In [ ]:
data = ERA.DataFile("telescope-data.uvfits",print_metadata=True)
#data = ERA.DataFile("data/example_bpcal.uvfits",print_metadata=True)
#data = ERA.DataFile("data/example_fullcal.uvfits",print_metadata=True)

### Figure: UV Coverage

In [ ]:
output_view.clear_output()
with output_view:
    display( data.fig_uv_coverage() )

### Figure: Visibilities vs Baseline Length

In [ ]:
output_view.clear_output()
with output_view:
    display( data.fig_vis_v_baseline() )

### Figure: All sky image

In [ ]:
output_view.clear_output()
with output_view:
    display( data.fig_all_sky_image() )

### Figure: Image vs Baseline Length (Increasing Resolution)

In [ ]:
output_view.clear_output()
with output_view:
    data.fig_img_v_baseline().show()

### Figure: Image & Visibilities vs Baseline Length

In [ ]:
output_view.clear_output()
with output_view:
    data.fig_imgvis_v_baseline().show()

### Figure: Image vs UV Coverage

In [ ]:
output_view.clear_output()
with output_view:
    data.fig_img_v_uv().show()